In [1]:
%aiida

Loaded AiiDA DB environment.

In [2]:
#from aiida.orm.data.base import Int, Float, Bool, Str, List
#from aiida.orm.data.structure import StructureData
#from aiida.orm.data.parameter import ParameterData
#from aiida.orm.data.folder import FolderData
#from aiida.work.run import submit

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import numpy as np

import tempfile
import shutil

import ase
import ase.io
from ase import Atom

from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, MetadataWidget, StructureBrowserWidget
from aiida.engine import submit, run, run_get_node

#from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details_dev import DFTDetails
from apps.surfaces.widgets.neb_details import NebDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
#from apps.surfaces.widgets.submit_button import SubmitButton


from ase.data import vdw_radii

from apps.surfaces.widgets import analyze_structure

NebWorkChain = WorkflowFactory('neb') 


_ColormakerRegistry()

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
## GENERAL PARAMETERS
style = {'description_width': '120px'}
layout = {'width': '70%'}
the_workchain=NebWorkChain
job_details = {'workchain':'NebWorkChain'}
atoms=[]
details=[]
slab_analyzed=[]

In [5]:
#from tempfile import NamedTemporaryFile
#from base64 import b64encode
#def render_thumbnail(atoms):
#    tmp = NamedTemporaryFile()
#    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
#    raw = open(tmp.name).read()
#    tmp.close()
#    return b64encode(raw)
#
#def display_thumbnail(th):
#    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [6]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global atoms,slab_analyzed
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    slab_analyzed = find_mol.analyze_slab(atoms)
    job_details['slab_analyzed']=slab_analyzed
    dft_details_widget.reset()  
    neb_details_widget.reset()
    #refresh_structure_view()
    guess_calc_params()
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params():
    method=dft_details_widget.calc_type.value
    valid_slab,msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_nodes=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,num_machines=num_nodes,calc_type=method)
        #dft_details_widget.fixed_atoms.value = atoms_to_fix
        #dft_details_widget.num_nodes.value = num_nodes
    else:
        print(msg)

# Submit a NEB calculation

## Structure

In [7]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

##VIEWER
viewer_widget = ViewerDetails()

mol_ids_info_out = ipw.Output()

display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

##CODE
computer_code_dropdown = CodeDropdown(input_plugin='cp2k', path_to_root="../../")

# Misc details
style = {'description_width': '120px'}
layout = {'width': '70%'}
num_nodes_selector = ipw.IntText(
    value=1,
    description='# Nodes',
    style=style, layout=layout)
calc_name_selector = ipw.Text(
    description='Calculation Name: ',
    placeholder='A great name.',
    style=style, layout=layout)

##DFT
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

##NEB
neb_details_widget = NebDetails(computer_code_dropdown, dft_details_widget) 

display(ipw.VBox([computer_code_dropdown, num_nodes_selector, calc_name_selector, dft_details_widget,neb_details_widget]))


## Submission

In [8]:
builder = NebWorkChain.get_builder()
builder.cp2k_code = computer_code_dropdown.selected_code
builder.structure = struct_browser.results.value
builder.max_force = Float(dft_details_widget.max_force.value)
builder.calc_type = Str(dft_details_widget.calc_type.value)
#builder.workchain = 
builder.vdw_switch = Bool(dft_details_widget.vdw_switch.value)
builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
builder.fixed_atoms = Str(dft_details_widget.fixed_atoms)
builder.num_machines = Int(num_nodes_selector.value)
builder.struc_folder = neb_details_widget.struct_folder
builder.wfn_cp_commands = neb_details_widget.aiida_wfn_cp_list
builder.calc_name = Str(calc_name_selector.value)
builder.nproc_rep = Int(neb_details_widget.proc_rep.value)
builder.nreplicas = Int(neb_details_widget.num_rep.value)
builder.spring = Float(neb_details_widget.spring_constant.value)
builder.rotate = Bool(neb_details_widget.rotate_frames.value)
builder.align = Bool(neb_details_widget.align_frames.value)
builder.nstepsit = Int(neb_details_widget.nsteps_it.value)
builder.endpoints = Bool(neb_details_widget.optimize_endpoints.value)
builder.replica_pks = Str(neb_details_widget.text_replica_pks.value)

ValueError: invalid attribute value value 'structure' is not of the right type. Got '<class 'bool'>', expected '<class 'aiida.orm.nodes.data.structure.StructureData'>'

In [ ]:
builder = NebWorkChain.get_builder()
builder.cp2k_code = computer_code_dropdown.selected_code
builder.structure = struct_browser.results.value
builder.max_force = Float(dft_details_widget.max_force.value)
builder.calc_type = Str(dft_details_widget.calc_type.value)
#builder.workchain = 
builder.vdw_switch = Bool(dft_details_widget.vdw_switch.value)
builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
builder.fixed_atoms = Str(dft_details_widget.fixed_atoms)
builder.num_machines = Int(num_nodes_selector.value)
builder.struc_folder = neb_details_widget.struct_folder
builder.wfn_cp_commands = neb_details_widget.aiida_wfn_cp_list
builder.calc_name = Str(calc_name_selector.value)
builder.nproc_rep = Int(neb_details_widget.proc_rep.value)
builder.nreplicas = Int(neb_details_widget.num_rep.value)
builder.spring = Float(neb_details_widget.spring_constant.value)
builder.rotate = Bool(neb_details_widget.rotate_frames.value)
builder.align = Bool(neb_details_widget.align_frames.value)
builder.nstepsit = Int(neb_details_widget.nsteps_it.value)
builder.endpoints = Bool(neb_details_widget.optimize_endpoints.value)
builder.replica_pks = Str(neb_details_widget.text_replica_pks.value)


builder.cp2k_code = load_node(975)
builder.structure = load_node(974)
builder.num_machines = Int(1)
builder.calc_name = Str("c2h2 test")
#builder.cell
builder.fixed_atoms = Str("")
builder.colvar_targets = Str("1.3 1.5 1.7 2.0")
builder.target_unit = Str('angstrom')
builder.spring = Float(30.0)
builder.spring_unit = Str('eV/angstrom^2')
builder.subsys_colvar = Dict(dict={'DISTANCE': {'ATOMS': '2 3 '}})
builder.calc_type = Str('Full DFT')
builder.mgrid_cutoff = Int(600)
builder.metadata = {
    "description": "c2h2 test",
    "label": "ReplicaWorkChain",
}
print("----")
print(builder)
print("----")

In [ ]:

#display submit button
btn_submit = SubmitButton(the_workchain=the_workchain,job_details=job_details)
display(btn_submit)